In [1]:
from utils import *
import copy
import random
from collections import Counter
import time
from collections import Counter, defaultdict
import bktree

In [2]:
tpcds_scansets, num_tpcds_tables = get_tpcds_scansets()
tpcds_scansets = set(tpcds_scansets)  # We don't care about duplicates in TPC-DS scansets

In [19]:
if False:
    redset_scansets = [(i, i+1, i+2) for i in range(1, 10)]
else:
    redset_scansets, num_redset_tables = get_redset_scansets(
        instance_id=135,
        user_id=None,
        database_id=0,
        query_type="select",
        limit=100000,
        min_num_tables=3
    )
    redset_scansets, redset_scanset_conversion = translate_scansets(redset_scansets)

In [20]:
# Count repetition of each scanset in the redset scansets
redset_scanset_counters = Counter(redset_scansets)

c_distinct_redset_scansets = list(set(redset_scansets)) # Remove duplicates
validate_scansets(c_distinct_redset_scansets)
c_redset_scanset_counters = [0] * len(c_distinct_redset_scansets)
for idx, scanset in enumerate(c_distinct_redset_scansets):
    c_redset_scanset_counters[idx] = redset_scanset_counters[scanset]
len(c_distinct_redset_scansets), len(c_redset_scanset_counters)

c_r_table_to_scansets = [[] for _ in range(num_redset_tables + 1)]
for idx, scanset in enumerate(c_distinct_redset_scansets):
    for r_table in scanset:
        c_r_table_to_scansets[r_table].append(idx)

print(f"We have {len(c_distinct_redset_scansets)} distinct redset scansets with {len(c_r_table_to_scansets)} r_tables.")

We have 23 distinct redset scansets with 173 r_tables.


In [21]:
import statistics
sum(redset_scanset_counters.values()), \
    statistics.fmean(map(len, redset_scansets)), \
    statistics.fmean(map(len, c_distinct_redset_scansets)), \
    statistics.median(map(len, redset_scansets)), \
    statistics.median(map(len, c_distinct_redset_scansets))

(239, 5.933054393305439, 8.782608695652174, 5, 7)

In [23]:
# TODO:!!! A plot with scanset size on x-axis and scanset repetition ratio would be really nice!!!

N_THREADS = 64
N_ITERATIONS_PER_THREAD = 4

start = time.time()
print(bktree.find_optimal_bijection(
    N_THREADS,
    N_ITERATIONS_PER_THREAD,
    # c_redset_scanset_counters,
    [1 for _ in range(len(c_distinct_redset_scansets))], # For a better interpretable distance metric
    c_distinct_redset_scansets,
    tpcds_scansets,
    c_r_table_to_scansets,
    num_redset_tables,
    num_tpcds_tables
))
time.time() - start

(58.0, {172: 8, 171: 6, 170: 23, 169: 1, 168: 13, 167: 20, 166: 9, 165: 4, 164: 8, 163: 3, 162: 8, 161: 11, 160: 12, 159: 18, 158: 12, 157: 15, 156: 8, 155: 5, 154: 17, 153: 10, 152: 9, 151: 7, 150: 13, 149: 4, 148: 3, 147: 18, 146: 16, 145: 6, 144: 1, 143: 18, 142: 12, 141: 17, 140: 16, 139: 8, 138: 5, 137: 3, 136: 10, 135: 7, 134: 6, 133: 13, 132: 9, 131: 4, 130: 13, 129: 7, 128: 12, 59: 8, 58: 6, 57: 5, 56: 12, 55: 8, 54: 18, 53: 14, 52: 12, 51: 5, 50: 4, 49: 8, 48: 23, 47: 15, 46: 24, 45: 23, 44: 1, 43: 8, 42: 7, 41: 22, 40: 7, 39: 1, 38: 9, 37: 22, 36: 6, 35: 5, 34: 3, 33: 13, 32: 10, 31: 19, 30: 16, 13: 19, 12: 14, 11: 3, 10: 16, 9: 14, 8: 2, 7: 12, 6: 4, 5: 12, 4: 11, 3: 11, 2: 8, 1: 4, 14: 6, 15: 11, 16: 6, 17: 8, 18: 20, 19: 12, 20: 4, 21: 7, 22: 18, 23: 5, 24: 6, 25: 8, 26: 9, 27: 3, 28: 12, 29: 17, 60: 17, 61: 3, 62: 2, 63: 4, 64: 16, 65: 22, 66: 14, 67: 23, 68: 1, 69: 19, 70: 18, 71: 19, 72: 24, 73: 5, 74: 8, 75: 6, 76: 24, 77: 16, 78: 13, 79: 15, 80: 14, 81: 17, 82: 18, 83

3.521334171295166

## Explore scansets in TPCDS

In [24]:
[scanset for scanset in tpcds_scansets if len(scanset) == 2]

[(14, 18), (12, 18)]

## Explore scansets in Redset

In [4]:
redset_scansets

[(25, 44),
 (2, 49),
 (2, 49),
 (2, 56),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (4, 10),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (54, 55),
 (54, 55),
 (54, 55),
 (2, 56),
 (2, 56),
 (2, 56),
 (2, 56),
 (2, 49, 56),
 (2, 49, 56),
 (2, 49, 56),
 (2, 49),
 (31, 32),
 (31, 32),
 (31, 32),
 (31, 32),
 (18, 30),
 (18, 30),
 (18, 30),
 (4, 28),
 (4, 28),
 (18, 45),
 (4, 18),
 (9, 18),
 (9, 18),
 (9, 18),
 (9, 18),
 (9, 18),
 (9, 18),
 (9, 18),
 (18, 29),
 (18, 29),
 (9, 18),
 (18, 29),
 (18, 29),
 (18, 51),
 (4, 50),
 (46, 52),
 (46, 52),
 (46, 52),
 (24, 40),
 (18, 26),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (7, 12),
 (1, 2),
 (2, 3),
 (2, 3),
 (11, 13